In [1]:
import numpy as np
from numba import njit, prange
from functions import is_target_blocked, v_of_fy

In [2]:
import yaml

with open('config.yaml', 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

M1_start_pos = np.array(config['M_start_pos'][0])
M2_start_pos = np.array(config['M_start_pos'][1])
M3_start_pos = np.array(config['M_start_pos'][2])

M_speed = np.array(config['M_speed'])
cloud_v = np.array(config['cloud_v'])
g = np.array(config['g'])

FY1_start_xy = np.array(config['FY_start_xy'][0])
FY2_start_xy = np.array(config['FY_start_xy'][1])
FY3_start_xy = np.array(config['FY_start_xy'][2])
FY4_start_xy = np.array(config['FY_start_xy'][3])
FY5_start_xy = np.array(config['FY_start_xy'][4])

FY1_height = np.array(config['FY_height'][0])
FY2_height = np.array(config['FY_height'][1])
FY3_height = np.array(config['FY_height'][2])
FY4_height = np.array(config['FY_height'][3])
FY5_height = np.array(config['FY_height'][4])

Vm1 = M_speed * -M1_start_pos / np.linalg.norm(M1_start_pos)
Vm2 = M_speed * -M2_start_pos / np.linalg.norm(M2_start_pos)
Vm3 = M_speed * -M3_start_pos / np.linalg.norm(M3_start_pos)

delta_t = 0.01

In [3]:
from scipy.optimize import differential_evolution

@njit(fastmath=True, parallel=True, cache=True, nogil=True)
def objective(x):
    theta1, speed1, t_111, t_112, t_121, t_122, t_131, t_132, theta2, speed2, t_211, t_212, t_221, t_222, t_231, t_232, theta3, speed3, t_311, t_312, t_321, t_322, t_331, t_332, theta4, speed4, t_411, t_412, t_421, t_422, t_431, t_432, theta5, speed5, t_511, t_512, t_521, t_522, t_531, t_532 = x = x
    V1 = v_of_fy(speed1, theta1)
    V2 = v_of_fy(speed2, theta2)
    V3 = v_of_fy(speed3, theta3)
    V4 = v_of_fy(speed4, theta4)
    V5 = v_of_fy(speed5, theta5)
    
    T1 = np.sqrt(np.sum(M1_start_pos**2)) / 300
    T2 = np.sqrt(np.sum(M2_start_pos**2)) / 300
    T3 = np.sqrt(np.sum(M3_start_pos**2)) / 300
    tmax = max(T1, T2, T3)
    steps = int(tmax / delta_t)

    cloud_center_11 = FY1_start_xy + FY1_height + V1 * (t_111 + t_112)+ 0.5 * g * t_112**2
    cloud_center_12 = FY1_start_xy + FY1_height + V1 * (t_111 + t_121 + t_122)+ 0.5 * g * t_122**2
    cloud_center_13 = FY1_start_xy + FY1_height + V1 * (t_111 + t_121 + t_131 + t_132)+ 0.5 * g * t_132**2
    cloud_center_21 = FY2_start_xy + FY2_height + V2 * (t_211 + t_212)+ 0.5 * g * t_212**2
    cloud_center_22 = FY2_start_xy + FY2_height + V2 * (t_211 + t_221 + t_222)+ 0.5 * g * t_222**2
    cloud_center_23 = FY2_start_xy + FY2_height + V2 * (t_211 + t_221 + t_231 + t_232) + 0.5 * g * t_232**2
    cloud_center_31 = FY3_start_xy + FY3_height + V3 * (t_311 + t_312) + 0.5 * g * t_312**2
    cloud_center_32 = FY3_start_xy + FY3_height + V3 * (t_311 + t_321 + t_322) + 0.5 * g * t_322**2
    cloud_center_33 = FY3_start_xy + FY3_height + V3 * (t_311 + t_321 + t_331 + t_332) + 0.5 * g * t_332**2
    cloud_center_41 = FY4_start_xy + FY4_height + V4 * (t_411 + t_412) + 0.5 * g * t_412**2
    cloud_center_42 = FY4_start_xy + FY4_height + V4 * (t_411 + t_421 + t_422) + 0.5 * g * t_422**2
    cloud_center_43 = FY4_start_xy + FY4_height + V4 * (t_411 + t_421 + t_431 + t_432) + 0.5 * g * t_432**2
    cloud_center_51 = FY5_start_xy + FY5_height + V5 * (t_511 + t_512) + 0.5 * g * t_512**2
    cloud_center_52 = FY5_start_xy + FY5_height + V5 * (t_511 + t_521 + t_522) + 0.5 * g * t_522**2
    cloud_center_53 = FY5_start_xy + FY5_height + V5 * (t_511 + t_521 + t_531 + t_532) + 0.5 * g * t_532**2
    counter = 0
    for i in prange(steps):
        t = i * delta_t
        M1_pos_now = M1_start_pos + Vm1 * t
        M2_pos_now = M1_start_pos + Vm2 * t
        M3_pos_now = M1_start_pos + Vm3 * t
        cloud_center_11_now = cloud_center_11 + cloud_v * (t - t_111 - t_112)
        cloud_center_12_now = cloud_center_12 + cloud_v * (t - t_111 - t_121 - t_122)
        cloud_center_13_now = cloud_center_13 + cloud_v * (t - t_111 - t_121 - t_131 - t_132)
        cloud_center_21_now = cloud_center_21 + cloud_v * (t - t_211 - t_212)
        cloud_center_22_now = cloud_center_22 + cloud_v * (t - t_211 - t_221 - t_222)
        cloud_center_23_now = cloud_center_23 + cloud_v * (t - t_211 - t_221 - t_231 - t_232)
        cloud_center_31_now = cloud_center_31 + cloud_v * (t - t_311 - t_312)
        cloud_center_32_now = cloud_center_32 + cloud_v * (t - t_311 - t_321 - t_322)
        cloud_center_33_now = cloud_center_33 + cloud_v * (t - t_311 - t_321 - t_331 - t_332)
        cloud_center_41_now = cloud_center_41 + cloud_v * (t - t_411 - t_412)
        cloud_center_42_now = cloud_center_42 + cloud_v * (t - t_411 - t_421 - t_422)
        cloud_center_43_now = cloud_center_43 + cloud_v * (t - t_411 - t_421 - t_431 - t_432)
        cloud_center_51_now = cloud_center_51 + cloud_v * (t - t_511 - t_512)
        cloud_center_52_now = cloud_center_52 + cloud_v * (t - t_511 - t_521 - t_522)
        cloud_center_53_now = cloud_center_53 + cloud_v * (t - t_511 - t_521 - t_531 - t_532)
        
        if (is_target_blocked(M1_pos_now, cloud_center_11_now, t - t_111 - t_112) + \
            is_target_blocked(M1_pos_now, cloud_center_12_now, t - t_111 - t_121 - t_122) + \
            is_target_blocked(M1_pos_now, cloud_center_13_now, t - t_111 - t_121 - t_131 - t_132) + \
            is_target_blocked(M1_pos_now, cloud_center_21_now, t - t_211 - t_212) + \
            is_target_blocked(M1_pos_now, cloud_center_22_now, t - t_211 - t_221 - t_222) + \
            is_target_blocked(M1_pos_now, cloud_center_23_now, t - t_211 - t_221 - t_231 - t_232) + \
            is_target_blocked(M1_pos_now, cloud_center_31_now, t - t_311 - t_312) + \
            is_target_blocked(M1_pos_now, cloud_center_32_now, t - t_311 - t_321 - t_322) + \
            is_target_blocked(M1_pos_now, cloud_center_33_now, t - t_311 - t_321 - t_331 - t_332) + \
            is_target_blocked(M1_pos_now, cloud_center_41_now, t - t_411 - t_412) + \
            is_target_blocked(M1_pos_now, cloud_center_42_now, t - t_411 - t_421 - t_422) + \
            is_target_blocked(M1_pos_now, cloud_center_43_now, t - t_411 - t_421 - t_431 - t_432) + \
            is_target_blocked(M1_pos_now, cloud_center_51_now, t - t_511 - t_512) + \
            is_target_blocked(M1_pos_now, cloud_center_52_now, t - t_511 - t_521 - t_522) + \
            is_target_blocked(M1_pos_now, cloud_center_53_now, t - t_511 - t_521 - t_531 - t_532) + \
            is_target_blocked(M2_pos_now, cloud_center_11_now, t - t_111 - t_112) + \
            is_target_blocked(M2_pos_now, cloud_center_12_now, t - t_111 - t_121 - t_122) + \
            is_target_blocked(M2_pos_now, cloud_center_13_now, t - t_111 - t_121 - t_131 - t_132) + \
            is_target_blocked(M2_pos_now, cloud_center_21_now, t - t_211 - t_212) + \
            is_target_blocked(M2_pos_now, cloud_center_22_now, t - t_211 - t_221 - t_222) + \
            is_target_blocked(M2_pos_now, cloud_center_23_now, t - t_211 - t_221 - t_231 - t_232) + \
            is_target_blocked(M2_pos_now, cloud_center_31_now, t - t_311 - t_312) + \
            is_target_blocked(M2_pos_now, cloud_center_32_now, t - t_311 - t_321 - t_322) + \
            is_target_blocked(M2_pos_now, cloud_center_33_now, t - t_311 - t_321 - t_331 - t_332) + \
            is_target_blocked(M2_pos_now, cloud_center_41_now, t - t_411 - t_412) + \
            is_target_blocked(M2_pos_now, cloud_center_42_now, t - t_411 - t_421 - t_422) + \
            is_target_blocked(M2_pos_now, cloud_center_43_now, t - t_411 - t_421 - t_431 - t_432) + \
            is_target_blocked(M2_pos_now, cloud_center_51_now, t - t_511 - t_512) + \
            is_target_blocked(M2_pos_now, cloud_center_52_now, t - t_511 - t_521 - t_522) + \
            is_target_blocked(M2_pos_now, cloud_center_53_now, t - t_511 - t_521 - t_531 - t_532) + \
            is_target_blocked(M3_pos_now, cloud_center_11_now, t - t_111 - t_112) + \
            is_target_blocked(M3_pos_now, cloud_center_12_now, t - t_111 - t_121 - t_122) + \
            is_target_blocked(M3_pos_now, cloud_center_13_now, t - t_111 - t_121 - t_131 - t_132) + \
            is_target_blocked(M3_pos_now, cloud_center_21_now, t - t_211 - t_212) + \
            is_target_blocked(M3_pos_now, cloud_center_22_now, t - t_211 - t_221 - t_222) + \
            is_target_blocked(M3_pos_now, cloud_center_23_now, t - t_211 - t_221 - t_231 - t_232) + \
            is_target_blocked(M3_pos_now, cloud_center_31_now, t - t_311 - t_312) + \
            is_target_blocked(M3_pos_now, cloud_center_32_now, t - t_311 - t_321 - t_322) + \
            is_target_blocked(M3_pos_now, cloud_center_33_now, t - t_311 - t_321 - t_331 - t_332) + \
            is_target_blocked(M3_pos_now, cloud_center_41_now, t - t_411 - t_412) + \
            is_target_blocked(M3_pos_now, cloud_center_42_now, t - t_411 - t_421 - t_422) + \
            is_target_blocked(M3_pos_now, cloud_center_43_now, t - t_411 - t_421 - t_431 - t_432) + \
            is_target_blocked(M3_pos_now, cloud_center_51_now, t - t_511 - t_512) + \
            is_target_blocked(M3_pos_now, cloud_center_52_now, t - t_511 - t_521 - t_522) + \
            is_target_blocked(M3_pos_now, cloud_center_53_now, t - t_511 - t_521 - t_531 - t_532)):
            counter += 1
            
    return -counter * delta_t

In [ ]:
bounds = [(0, 2*np.pi), (70, 140), (0, 20), (0, 20), (1, 20), (0, 20), (1, 20), (0, 20),
          (0, 2*np.pi), (70, 140), (0, 20), (0, 20), (1, 20), (0, 20), (1, 20), (0, 20),
          (0, 2*np.pi), (70, 140), (0, 20), (0, 20), (1, 20), (0, 20), (1, 20), (0, 20),
          (0, 2*np.pi), (70, 140), (0, 20), (0, 20), (1, 20), (0, 20), (1, 20), (0, 20),
          (0, 2*np.pi), (70, 140), (0, 20), (0, 20), (1, 20), (0, 20), (1, 20), (0, 20)]
result = differential_evolution(objective, bounds, popsize=64, mutation=(0.5, 1.0), recombination=0.7, seed=42, polish=True, disp=True, maxiter=5000)
theta1, speed1, t_111, t_112, t_121, t_122, t_131, t_132, theta2, speed2, t_211, t_212, t_221, t_222, t_231, t_232, theta3, speed3, t_311, t_312, t_321, t_322, t_331, t_332, theta4, speed4, t_411, t_412, t_421, t_422, t_431, t_432, theta5, speed5, t_511, t_512, t_521, t_522, t_531, t_532 = result.x
max_time = -result.fun
print(f"最长遮蔽时间: {max_time:.2f} s")
print(f"最优参数: theta1={theta1:.2f}, speed1={speed1:.2f}, t_111={t_111:.2f}, t_112={t_112:.2f}, t_121={t_121:.2f}, t_122={t_122:.2f}, t_131={t_131:.2f}, t_132={t_132:.2f}, theta2={theta2:.2f}, speed2={speed2:.2f}, t_211={t_211:.2f}, t_212={t_212:.2f}, t_221={t_221:.2f}, t_222={t_222:.2f}, t_231={t_231:.2f}, t_232={t_232:.2f}, theta3={theta3:.2f}, speed3={speed3:.2f}, t_311={t_311:.2f}, t_312={t_312:.2f}, t_321={t_321:.2f}, t_322={t_322:.2f}, t_331={t_331:.2f}, t_332={t_332:.2f}, theta4={theta4:.2f}, speed4={speed4:.2f}, t_411={t_411:.2f}, t_412={t_412:.2f}, t_421={t_421:.2f}, t_422={t_422:.2f}, t_431={t_431:.2f}, t_432={t_432:.2f}, theta5={theta5:.2f}, speed5={speed5:.2f}, t_511={t_511:.2f}, t_512={t_512:.2f}, t_521={t_521:.2f}, t_522={t_522:.2f}, t_531={t_531:.2f}, t_532={t_532:.2f}")

differential_evolution step 1: f(x)= -4.0200000000000005
differential_evolution step 2: f(x)= -4.23
differential_evolution step 3: f(x)= -4.23
differential_evolution step 4: f(x)= -4.3
differential_evolution step 5: f(x)= -4.55
differential_evolution step 6: f(x)= -4.55
differential_evolution step 7: f(x)= -4.55
differential_evolution step 8: f(x)= -4.55
differential_evolution step 9: f(x)= -4.55
differential_evolution step 10: f(x)= -4.55
differential_evolution step 11: f(x)= -4.55
differential_evolution step 12: f(x)= -4.57
differential_evolution step 13: f(x)= -4.57
differential_evolution step 14: f(x)= -4.58
differential_evolution step 15: f(x)= -4.58
differential_evolution step 16: f(x)= -4.58
differential_evolution step 17: f(x)= -4.58
differential_evolution step 18: f(x)= -4.58
differential_evolution step 19: f(x)= -4.58
differential_evolution step 20: f(x)= -4.58
differential_evolution step 21: f(x)= -4.58
differential_evolution step 22: f(x)= -4.58
differential_evolution step 

In [ ]:
from functions import fy_bomb_position, fy_throw_position

@njit(fastmath=True, cache=True, nogil=True)
def calc_interference(M_start_pos, Vm, FY_start_xy, FY_height, theta, speed, t_throw, t_delay, g, cloud_v, delta_t):
    V = v_of_fy(speed, theta)
    throw_pos = fy_throw_position(FY_start_xy, FY_height, theta, speed, t_throw)
    bomb_pos = fy_bomb_position(FY_start_xy, FY_height, theta, speed, t_throw, t_delay, g)
    cloud_center = bomb_pos
    M_end_pos = M_start_pos + Vm * (t_throw + t_delay)
    T = np.linalg.norm(M_end_pos) / 300
    tmax = max(20.0, T)
    steps = int(tmax / delta_t)
    counter = 0
    for i in prange(steps):
        t = i * delta_t
        M_pos_now = M_end_pos + Vm * t
        cloud_center_now = cloud_center + cloud_v * t
        if is_target_blocked(M_pos_now, cloud_center_now, t):
            counter += 1
    return throw_pos, bomb_pos, counter * delta_t

smoke_params = [
    (FY1_start_xy, FY1_height, theta1, speed1, t_111, t_112),
    (FY1_start_xy, FY1_height, theta1, speed1, t_111 + t_121, t_122),
    (FY1_start_xy, FY1_height, theta1, speed1, t_111 + t_121 + t_131, t_132),
    (FY2_start_xy, FY2_height, theta2, speed2, t_211, t_212),
    (FY2_start_xy, FY2_height, theta2, speed2, t_211 + t_221, t_222),
    (FY2_start_xy, FY2_height, theta2, speed2, t_211 + t_221 + t_231, t_232),
    (FY3_start_xy, FY3_height, theta3, speed3, t_311, t_312),
    (FY3_start_xy, FY3_height, theta3, speed3, t_311 + t_321, t_322),
    (FY3_start_xy, FY3_height, theta3, speed3, t_311 + t_321 + t_331, t_332),
    (FY4_start_xy, FY4_height, theta4, speed4, t_411, t_412),
    (FY4_start_xy, FY4_height, theta4, speed4, t_411 + t_421, t_422),
    (FY4_start_xy, FY4_height, theta4, speed4, t_411 + t_421 + t_431, t_432),
    (FY5_start_xy, FY5_height, theta5, speed5, t_511, t_512),
    (FY5_start_xy, FY5_height, theta5, speed5, t_511 + t_521, t_522),
    (FY5_start_xy, FY5_height, theta5, speed5, t_511 + t_521 + t_531, t_532),
]

missiles = [
    (M1_start_pos, Vm1, "M1"),
    (M2_start_pos, Vm2, "M2"),
    (M3_start_pos, Vm3, "M3"),
]

for idx, (FY_start_xy, FY_height, theta, speed, t_throw, t_delay) in enumerate(smoke_params, 1):
    for M_start_pos, Vm, mname in missiles:
        throw_pos, bomb_pos, interference_time = calc_interference(
            M_start_pos, Vm, FY_start_xy, FY_height, theta, speed, t_throw, t_delay, g, cloud_v, delta_t
        )
        
        print(f"烟幕弹{idx} 投放位置: {throw_pos}, 爆炸位置: {bomb_pos}, 干扰导弹: {mname}, 干扰时长: {interference_time:.2f} s")